In [1]:
from support_module import ImageToNumDataset, NoMaskModel

In [2]:
import torch
from torchvision.transforms.v2 import ToTensor, ToDtype, Normalize, Compose

transform = Compose([
    ToTensor(),
    ToDtype(torch.float32, scale=True),
    Normalize((0.5,), (0.5,))
])

/Users/daniilsuhanov/DataspellProjects/ML-intensive-Yandex-Academy-autumn-2023/venv/lib/python3.10/site-packages/torchvision/transforms/v2/_deprecated.py:43: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.
  warnings.warn(


In [3]:
dataset = ImageToNumDataset("data/train_images", answers_file="data/train_answers.csv", transform=transform)

In [4]:
from torch.utils.data import DataLoader, random_split

train_dataset, validation_dataset = random_split(dataset, (0.8, 0.2))

train_dataloader = DataLoader(train_dataset, batch_size=2**10, shuffle=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=2**10, shuffle=False)

In [5]:
model = NoMaskModel()

In [6]:
from torch import nn, optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [7]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE

device(type='cpu')

In [8]:
from ignite.metrics import Accuracy, Loss
from ignite.engine import create_supervised_trainer, create_supervised_evaluator

trainer = create_supervised_trainer(model, optimizer, criterion, device=DEVICE)
evaluator = create_supervised_evaluator(model, metrics={'accuracy': Accuracy(), 'nll': Loss(criterion)}, device=DEVICE)

In [9]:
# Сбор потерь и метрик для построения графиков
train_loss_values = []
validation_loss_values = []
validation_accuracy_values = []

In [10]:
import logging


logging.basicConfig(level=logging.INFO)

In [11]:
EPOCHS = 2
EPOCHS_VALIDATION = 5

In [12]:
from ignite.engine import Events


@trainer.on(Events.EPOCH_COMPLETED)
def log_training_results(engine):
    # Запуск оценки модели на обучающем наборе данных
    evaluator.run(train_dataloader)
    metrics = evaluator.state.metrics
    # Сбор и вывод средней точности и потерь на обучающем наборе
    train_loss_values.append(metrics['nll'])
    logging.info(
        f"Training Results - Epoch: {engine.state.epoch}  "
        f"Avg accuracy: {metrics['accuracy']:.2f} "
        f"Avg loss: {metrics['nll']:.2f}"
    )

@trainer.on(Events.EPOCH_COMPLETED)
def log_validation_results(engine):
    logging.info(f"Starting validation on epoch {engine.state.epoch}")
    # Запуск оценки модели на валидационном наборе данных
    evaluator.run(validation_dataloader, max_epochs=EPOCHS_VALIDATION)
    metrics = evaluator.state.metrics
    # Сбор и вывод средней точности и потерь на валидационном наборе
    validation_loss_values.append(metrics['nll'])
    validation_accuracy_values.append(metrics['accuracy'])
    logging.info(
        f"Validation Results - Epoch: {engine.state.epoch}  "
        f"Avg accuracy: {metrics['accuracy']:.3f} "
        f"Avg loss: {metrics['nll']:.3f}"
    )
    logging.info(f"End of validation on epoch {engine.state.epoch}")

In [ ]:
trainer.run(train_dataloader, max_epochs=EPOCHS)

INFO:ignite.engine.engine.Engine:Engine run starting with max_epochs=2.
INFO:root:Starting conv1...


In [ ]:
from matplotlib import pyplot as plt

# Графики обучения
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(train_loss_values, label='Training Loss')
plt.plot(validation_loss_values, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(validation_accuracy_values, label='Validation Accuracy', color='red')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
model.eval()

plt.figure(figsize=(15,4))
# Смотрим результат обучения
for i in range(20):
    x, y_true = validation_dataset[i]
    y_pred = torch.argmax(model(x.to(DEVICE)))
    plt.subplot(2, 10, i + 1)
    plt.imshow(x.permute(1, 2, 0) * 0.25 + 0.5)
    plt.xticks([])
    plt.yticks([])
    class_to_idx = {value: key for key, value in validation_dataset.class_to_idx.items()}
    plt.title(f'True = {class_to_idx[y_true]}\nPred = {class_to_idx[y_pred.item()]}')

plt.show()